## Speckgürtel-Projekt

### Import

In [1]:
import pandas as pd
import numpy as np
import warnings
import json
import os
import glob
from sklearn.preprocessing import StandardScaler

warnings.simplefilter(action='ignore', category=FutureWarning)

### Load

In [5]:
# Load the CSV file into a DataFrame
df = pd.read_csv('../data/Gemergte Daten vorläufig.csv')

df.columns = [
    'id', 'name', 'metropole', 'preis', 'angebot', 
    'preis_entwicklung', 'angebot_entwicklung', 
    'autobahn', 'zug', 'supermarkt', 
    'pendler', 'schule', 'einwohner'
]
df["id"]=df["id"].astype(str).str.zfill(5)
# Konvertierung von km in m
columns_to_convert = ['autobahn', 'zug', 'supermarkt']
df[columns_to_convert] = df[columns_to_convert] * 1000

# Display the first few rows of the DataFrame to verify
df.head(5)

,id,name,metropole,preis,angebot,preis_entwicklung,angebot_entwicklung,autobahn,zug,supermarkt,pendler,schule,einwohner
0,09173,Bad Tölz-Wolfratshausen,München,5689,391.0,0.067154,-0.055556,12550.0,4370.0,2150.0,16.863591,75.390603,7.560664
1,12060,Barnim,Berlin,3317,535.0,-0.028697,-0.256944,6810.0,2960.0,2580.0,41.134607,62.160062,11.307039
2,11000,Berlin,Berlin,5527,13095.0,0.008577,-0.259960,4470.0,1840.0,590.0,2.972945,58.087855,8.507855
3,05314,Bonn,Köln,3860,948.0,0.041273,-0.213278,2360.0,2400.0,720.0,13.774530,91.426461,3.811611
4,15084,Burgenlandkreis,Leipzig,1207,353.0,-0.009844,-0.386087,13390.0,3870.0,3450.0,1.501034,60.392608,-5.838318


### Standardize

Wir standisieren, um den Einfluss aller Features auf den Score anzugleichen. 

In [6]:
# Spalten, die standardisiert werden sollen
columns_to_standardize = [
    'preis', 'angebot', 
    'preis_entwicklung', 'angebot_entwicklung', 
    'autobahn', 'zug', 'supermarkt', 
    'pendler', 'schule', 'einwohner'
]

# DataFrame kopieren
df_scaled = df.copy()

# Standardisierung durchführen
scaler = StandardScaler()
df_scaled[columns_to_standardize] = scaler.fit_transform(df[columns_to_standardize])

# Ergebnis anzeigen
df_scaled.head(5)

,id,name,metropole,preis,angebot,preis_entwicklung,angebot_entwicklung,autobahn,zug,supermarkt,pendler,schule,einwohner
0,09173,Bad Tölz-Wolfratshausen,München,1.427611,-0.551858,1.356903,2.887122,1.253145,0.928451,0.542582,-0.276065,0.814740,0.552805
1,12060,Barnim,Berlin,-0.323231,-0.462676,-1.637596,-0.042280,0.019196,-0.179864,1.015658,1.562459,-0.363979,1.535295
2,11000,Berlin,Berlin,1.308034,7.315987,-0.473129,-0.086150,-0.483843,-1.060228,-1.173695,-1.328278,-0.726775,0.801206
3,05314,Bonn,Köln,0.077573,-0.206897,0.548349,0.592892,-0.937437,-0.620046,-1.030672,-0.510061,2.243387,-0.430388
4,15084,Burgenlandkreis,Leipzig,-1.880683,-0.575392,-1.048614,-1.920786,1.433723,0.535432,1.972813,-1.439775,-0.521443,-2.961091


### Save

Die Datei soll an das Frontend geliefert werden. Ebenfalls enthalten sind Spalten-Mediane und -Varianzen zur späteren Standardisierung des Userinputs.  

In [4]:
# Verzeichnis mit den JSON-Dateien
input_directory = 'data/city/'
output_directory = 'export/'

# Lade alle JSON-Dateien, die dem Muster data_[].json entsprechen
json_files = glob.glob(os.path.join(input_directory, 'data_*.json'))

# Iteriere über alle JSON-Dateien
for json_file in json_files:
    print(json_file)
    # Lade die Daten aus der aktuellen JSON-Datei
    with open(json_file, 'r') as file:
        city_data = json.load(file)

    # Extrahiere die Metropole (city) aus den Daten
    target_city = city_data['city']

    # Filtere die Kreise, die zur Metropole gehören
    filtered_df = df[df['metropole'] == target_city]  # Originalwerte
    filtered_df_scaled = df_scaled[df_scaled['metropole'] == target_city]  # Skalierte Werte

    kreise = []
for (_, row_original), (_, row_scaled) in zip(filtered_df.iterrows(), filtered_df_scaled.iterrows()):
    kreis_data = {
        ### Hier sollten die Ids nicht als Integer einfließen, damit Zeropaddings stattfindne
        "ags": str(row_original['id']).zfill(5),  # Verwende die ID als AGS
        "label": row_original['name'],  # Kreisname
        "data": []
    }
    for feature in ['preis', 'angebot', 'preis_entwicklung', 'angebot_entwicklung', 
                    'autobahn', 'zug', 'supermarkt', 'pendler', 'schule', 'einwohner']:
        kreis_data["data"].append({
            "id": feature,
            "displayValue": row_original[feature],  # Originalwert aus df
            "score": row_scaled[feature]  # Standardisierter Wert aus df_scaled
        })
    kreise.append(kreis_data)

# Erstelle die globalen Mittelwerte und Varianzen
### Das sind dann die Means und Variances über alle Städte oder die jeweilige Stadt?
city_means = {
    feature: scaler.mean_[columns_to_standardize.index(feature)]
    for feature in ['preis', 'angebot', 'preis_entwicklung', 'angebot_entwicklung', 
                    'autobahn', 'zug', 'supermarkt', 'pendler', 'schule', 'einwohner']
}

city_variances = {
    feature: scaler.var_[columns_to_standardize.index(feature)]
    for feature in ['preis', 'angebot', 'preis_entwicklung', 'angebot_entwicklung', 
                    'autobahn', 'zug', 'supermarkt', 'pendler', 'schule', 'einwohner']
}

# Erstelle die finale JSON-Struktur, inklusive zusätzlicher Werte aus der aktuellen JSON-Datei
output_data = {
    "city": target_city,
    "cityAGS": str(city_data.get("cityAGS")).zfill(5),  # Übernehme cityAGS
    "map": city_data.get("map"),  # Übernehme map-Daten
    "means": city_means,  # Globale Mittelwerte
    "variances": city_variances,  # Globale Varianzen 
    "kreise": kreise
}

# Speichere die JSON-Datei
output_file = os.path.join(output_directory, f'data_{target_city.lower()}.json')
os.makedirs(output_directory, exist_ok=True) 
with open(output_file, 'w') as file:
    json.dump(output_data, file, indent=4)

print(f"JSON-Datei für {target_city} wurde erfolgreich unter {output_file} gespeichert.")

data/city/data_leipzig.json
JSON-Datei für Leipzig wurde erfolgreich unter export/data_leipzig.json gespeichert.


### Calculation

Hier wird eine später im Frontend stattfindende Berechnung des Scores aufgrund der User-Werte simuliert. Es dient als Anschauungsbeispiel für die Frontend-Programierung.

##### User Input

Hier wird der User-Input defininert, wobei drei Inputtypen unterschieden werden: kontinuierliche Werte (Preis), kategoriale Werte (Angebot, Schule, Pendler) und Boolean-Werte (Autobahn, Zug, Supermarkt). 

Erklärung: "None" bedeutet, dass diese Variable vom User nicht gesetzt wurde und in der Berechnung nicht berücksichtigt wird. Kontinuum-Werte können jeden erdenklichen Wert annehmen. Kategorie-Werte könenn -1, 0 und 1 sein, die für die klickbaren Label im Interface stehen (z.B. "fallend", "neutral" und "steigend"). Boolean-Werte sind entweder "true" oder None. 

In [5]:
# User-Inputs definieren
user_inputs = {
    'preis': 1000,  # integriert (Werte: Kontinuum)
    'angebot': -1,  # integriert (Werte: Kategorie / -1, 0, 1)
    'preis_entwicklung': None,  # integriert (Werte: Kategorie / -1, 0, 1)
    'angebot_entwicklung': None,  # integriert (Werte: Kategorie / -1, 0, 1)
    'autobahn': None,  # integriert (Werte: Boolean / true, false)
    'zug': None,  # integriert (Werte: Boolean / true, false)
    'supermarkt': True,  # integriert (Werte: Boolean / true, false)
    'pendler': None,  # integriert (Werte: Kategorie / -1, 0, 1)
    'schule': None,  # integriert (Werte: Kategorie / -1, 0, 1)
    'einwohner': None  # integriert (Werte: Kategorie / -1, 0, 1)
}

# Anzahl der gültigen Inputs berechnen
valid_inputs = {key: value for key, value in user_inputs.items() if value is not None}
anzahl_der_inputs = len(valid_inputs)

##### Scaling of User Input

Bei kontinuerlichen Inputs müssen die User-Inputs komplett mit den Varianz-Werten der vorherigen Skalierung skaliert werden. Die dafür notwendigen Werte müssen dem JSON entnommen werden. Kategoriale Inputs und Boolean-Werte dürfen nicht skaliert werden. 

In [6]:

scaled_inputs = {}
for column, user_value in valid_inputs.items():
    if column in ['autobahn', 'zug', 'supermarkt']:
        # Boolean-Werte werden in True/False umgewandelt
        scaled_inputs[column] = user_value
    elif column in columns_to_standardize and column =="preis":
        # Index der Spalte im StandardScaler
        column_index = columns_to_standardize.index(column)
        # Skalierung des User-Inputs mit den gespeicherten Mittelwerten und Varianzen
        scaled_inputs[column] = (user_value - scaler.mean_[column_index]) / np.sqrt(scaler.var_[column_index])
    else:
        # Kategoriale Werte direkt übernehmen (unskaliert)
        scaled_inputs[column] = user_value

# Ergebnis anzeigen
print("Skalierte User-Inputs:")
for column, scaled_value in scaled_inputs.items():
    print(f"{column}: {scaled_value}")

Skalierte User-Inputs:
preis: -2.0334756239167926
angebot: -1
supermarkt: True


##### Score Calculation & Normalization

Bei der Berechnung des Scores wird nach den drei Input-Typen unterschieden. Im Anschluss wird der Score normiert, wobei 1 der Bestwert ist, um eine Vergleichbarkeit zu ermöglichen. 

In [7]:
dict_for_calc={"id":df_scaled["id"].tolist(),"name":df_scaled["name"].tolist()}
for key in scaled_inputs:
    scaled_user_input=scaled_inputs[key]
    #Kategorial
    if key in ['angebot', 'preis_entwicklung', 'angebot_entwicklung', 'schule', 'pendler', 'einwohner']:
        if scaled_user_input==0:
            ###Ist das hier korrekt? Wieso brauche wir keinen Bruch beim Median?
            dict_for_calc[key]=(abs(df_scaled[key] - df_scaled[key].median())).tolist()
        if scaled_user_input==-1:
            dict_for_calc[key]=((df_scaled[key] - df_scaled[key].min()) / -(df_scaled[key].min())).tolist()
        if scaled_user_input==1:
            dict_for_calc[key]=((df_scaled[key] - df_scaled[key].max()) / -(df_scaled[key].max())).tolist()

    if key in ['autobahn', 'zug', 'supermarkt']:
        if scaled_user_input==True:
            dict_for_calc[key]=((df_scaled[key].min() - df_scaled[key]) / df_scaled[key].min()).tolist() 
    if key == 'preis':
        dict_for_calc[key] = abs((df_scaled[key] - scaled_user_input)) / -scaled_user_input

df_calc=pd.DataFrame(dict_for_calc)

In [8]:
df_calc["score"]=1-(df_calc[df_calc.columns[2:]].sum(axis=1))/anzahl_der_inputs

In [9]:
# Min-Max-Normalisierung des Scores
score_min = df_calc['score'].min()
score_max = df_calc['score'].max()
df_calc['score_normalized'] = (df_calc['score'] - score_min) / (score_max - score_min)

original_params=list(set(valid_inputs.keys()))
col_list=["id","name"]
col_list.extend(original_params)
col_list.extend(['score', 'score_normalized'])
# Ergebnis anzeigen
df_calc[col_list].head()

,id,name,preis,angebot,supermarkt,score,score_normalized
0,09173,Bad Tölz-Wolfratshausen,1.702055,0.055139,1.445578,-0.067591,0.828760
1,12060,Barnim,0.841045,0.207832,1.834077,0.039015,0.850843
2,11000,Berlin,1.643250,13.526029,0.036139,-4.068473,0.000000
3,05314,Bonn,1.038148,0.645763,0.153593,0.387499,0.923029
4,15084,Burgenlandkreis,0.075139,0.014845,2.620111,0.096635,0.862779
